In [23]:
import time as t
import datetime as dt
import pandas as pd
import numpy as np
import sys
sys.path.insert(1, '../')
from utils.common import *

##loading scheduler
from apscheduler.schedulers.background import BackgroundScheduler
from apscheduler.jobstores.sqlalchemy import SQLAlchemyJobStore
jobstores = {
    'default': SQLAlchemyJobStore(url='sqlite:///../dbs/jobs.sqlite')
}
scheduler = BackgroundScheduler(jobstores=jobstores, timezone="UTC")
scheduler.start()


In [96]:
with SQLiteDB('../dbs/calls.db') as conn:
    query = "SELECT * FROM calls "  #WHERE launched is NULL
    df = pd.read_sql_query(query, conn)
    df = df.loc[dt.datetime.now() > pd.to_datetime(df['expected_launch_time_ts']) + dt.timedelta(minutes=5)]
df

,id,date,address,expected_launch_time,expected_launch_time_ts,s_mm2,s_ma2,s_fa2,s_q,s_sni,mcap_num,liq_num,launched,launch_time,buy,buy_time,buy_price
0,215618,2024-04-18 14:35:57,CfQSSRk6Md2cnZCzkJu2QjAVua5FzGMCua7CrhQGRURK,In 1 minute,2024-04-18 14:36:57,0,0,0,2,0,9050,18110,1,2024-04-18 14:37:00,None,None,None
1,215619,2024-04-18 14:37:11,GKMRt3xZznGK4wxoBw3SZfMTwVR9LLFTJxbqMCUvMabg,11 seconds ago,2024-04-18 14:37:00,1,0,0,0,2,1390,2500,1,2024-04-18 14:37:00,None,None,None
2,215620,2024-04-18 14:37:58,66Yp2kKFLsEjszAMVttwHFDZRZRyGt9kbKoXnuv99Dx3,36 seconds ago,2024-04-18 14:37:22,1,0,0,0,4,1730,1390,1,2024-04-18 14:42:00,None,None,None
3,215621,2024-04-18 14:38:20,Ecz4mak5jScXrwZkMGJB3mqqpeuJPWPTddJW6Rgmt9Za,27 seconds ago,2024-04-18 14:37:53,0,0,0,0,2,1110,1110,1,2024-04-18 14:38:00,None,None,None
4,215622,2024-04-18 14:39:15,BdSSmAmWAE8F2yhjAPW91Boa4fQfu98EKt3xxgLKQVxn,36 seconds ago,2024-04-18 14:38:39,1,1,0,0,2,40230,80460,1,2024-04-18 14:39:00,None,None,None
5,218927,2024-04-21 01:14:29,HbzH8fFYQRLjHgDK7G7qXYzWdV4fnwTSdb71zAZvJyDS,14 seconds ago,2024-04-21 01:14:15,0,0,0,2,0,1650,3290,1,2024-04-21 01:14:00,None,None,None
6,218928,2024-04-21 01:14:51,6xPz9dztCA6TdMsPZ6AHAZ6NEdGMjrouLb3fov8vbBJF,476017 hours and 14 minutes ago,2024-04-21 01:14:51,0,0,0,0,1,770,1500,1,2024-04-21 01:14:00,None,None,None
7,218929,2024-04-21 01:16:28,6XejiuxHwr31q7iZr4bYWzuNzzKoQWh4X4rrYP1qkt6t,13 seconds ago,2024-04-21 01:16:15,1,1,0,0,3,1490,2990,1,2024-04-21 01:16:00,None,None,None
8,218930,2024-04-21 01:17:25,7VEqMtsR2bRFU3g5cMTHuCYHdR6x1cwoVfoQ1uenJVqs,476017 hours and 17 minutes ago,2024-04-21 01:17:25,0,0,0,2,0,57080,23620,1,2024-04-21 01:17:00,None,None,None
9,218931,2024-04-21 01:17:26,9irAyMcsQSemhFiiSCZi6gFyQVsSWCKZgQT4MFd3ocdV,476017 hours and 17 minutes ago,2024-04-21 01:17:26,0,0,0,2,0,57080,23620,1,2024-04-21 01:17:00,None,None,None


In [ ]:
def update_launch_date(index, status, time):
    ltime = f'"{time}"' if time else "NULL"
    with SQLiteDB('../dbs/calls.db') as conn:
        update_statement = f"""
        UPDATE calls
        SET launched = {status}, launch_time = {ltime}
        WHERE id = {index}
        """
        #print(update_statement)
        conn.execute(update_statement)

In [120]:
for i,row in df[-2:].iterrows():
    print('trying: ',i, row['address'])
    elt = int(dt.datetime.strptime(row['expected_launch_time_ts'],'%Y-%m-%d %H:%M:%S').timestamp())
    tmp_df = get_price_data(row['address'], elt - 60*60, elt + 60*60*25)
    if not tmp_df.empty:
        launched = True
        launch_time = dt.datetime.fromtimestamp(tmp_df['unixTime'].min(), tz=dt.timezone.utc)
        scheduler.add_job(func = core_task,
                          trigger='date',
                          run_date=str(launch_time + dt.timedelta(minutes=20)),
                          id=str(row['id']),
                          jobstore='default',
                          kwargs={'token':row['address'], 'launch_time':launch_time})
    else:
        launched = False
        launch_time = None
    update_launch_date(row['id'], launched, launch_time)

trying:  5 HbzH8fFYQRLjHgDK7G7qXYzWdV4fnwTSdb71zAZvJyDS


In [121]:
scheduler.print_jobs()

Pending jobs:
    core_task (trigger: date[2024-04-21 01:34:00 UTC], pending)
    core_task (trigger: date[2024-04-21 01:34:00 UTC], pending)


In [163]:
with SQLiteDB('../dbs/jobs.sqlite') as conn:
    query = "SELECT * FROM apscheduler_jobs"
    df = pd.read_sql_query(query, conn)
df

,id,next_run_time,job_state


In [164]:
jobstores = {
    'default': SQLAlchemyJobStore(url='sqlite:///../dbs/jobs.sqlite')
}
scheduler = BackgroundScheduler(jobstores=jobstores, timezone="UTC")
scheduler.start()
scheduler.print_jobs()

Jobstore default:
    No scheduled jobs
